## train流程
## upernet

In [ ]:
# System libs
import os
import time
# import math
import random
# Numerical libs
import numpy as np
import torch
import torch.nn as nn


## main
这里是已经封装好的main代码

In [ ]:
def main(args):
    builder = ModelBuilder()
    # 载入网络结构
    net_encoder = builder.build_encoder(
        arch = args.arch_encoder,
        fc_dim = args.fc_dim,
        weights = args.weights_encoder
    )